In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import weibull_min
import matplotlib.pyplot as plt

df = pd.read_excel(r"D:\D\University\4th year\Thesis\Processed Data\DataMaster.xlsx", sheet_name="Breakdown")
# Combine dates and times correctly
df["BreakdownDatetime"] = df.apply(lambda row: pd.Timestamp.combine(row["BreakdownDate"], row["BreakdownTime"]), axis=1)
df["CompleteDatetime"]  = df.apply(lambda row: pd.Timestamp.combine(row["BreakdownDate"], row["CompleteTime"]), axis=1)

df["Downtime"] = (df["CompleteDatetime"] - df["BreakdownDatetime"]).dt.total_seconds() / 60  # Downtime in minutes
df["Uptime"]   = df["BreakdownDatetime"].shift(-1) - df["CompleteDatetime"]
df["Uptime"]   = df["Uptime"].dt.total_seconds() / 60  # Uptime in minutes
df = df.dropna(subset=["Uptime"])  # Remove the last NaN value

# Fit the uptime and downtime data to Weibull distribution
shape_u, loc_u, scale_u = weibull_min.fit(df["Uptime"], floc=0)
shape_d, loc_d, scale_d = weibull_min.fit(df["Downtime"], floc=0)

# Generate fitted distribution for uptime
x_uptime = np.linspace(df["Uptime"].min(), df["Uptime"].max(), 100)
fitted_pdf_uptime = weibull_min.pdf(x_uptime, shape_u, loc_u, scale_u)

# Generate fitted distribution for downtime
x_downtime = np.linspace(df["Downtime"].min(), df["Downtime"].max(), 100)
fitted_pdf_downtime = weibull_min.pdf(x_downtime, shape_d, loc_d, scale_d)

# Create subplots with one on the left and one on the right
fig, axs = plt.subplots(1, 2, figsize=(16, 6))

# Plot uptime distribution
axs[0].hist(df["Uptime"], bins=20, density=True, alpha=0.6, color='lightblue', label='Uptime Data')
axs[0].plot(x_uptime, fitted_pdf_uptime, 'r-', lw=2, label=f'Weibull fit\nShape: {shape_u:.2f}, Scale: {scale_u:.2f}')
axs[0].set_xlabel('Uptime (minutes)')
axs[0].set_ylabel('Density')
axs[0].set_title('Uptime Distribution and Weibull Fit')
axs[0].legend()

# Plot downtime distribution
axs[1].hist(df["Downtime"], bins=20, density=True, alpha=0.6, color='lightblue', label='Downtime Data')
axs[1].plot(x_downtime, fitted_pdf_downtime, 'r-', lw=2, label=f'Weibull fit\nShape: {shape_d:.2f}, Scale: {scale_d:.2f}')
axs[1].set_xlabel('Downtime (minutes)')
axs[1].set_ylabel('Density')
axs[1].set_title('Downtime Distribution and Weibull Fit')
axs[1].legend()

# Show the plot
plt.tight_layout()
plt.show()

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import weibull_min

df = pd.read_excel(r"D:\D\University\4th year\Thesis\Processed Data\DataMaster.xlsx", sheet_name="Breakdown")
# Combine dates and times correctly
df["BreakdownDatetime"] = df.apply(lambda row: pd.Timestamp.combine(row["BreakdownDate"], row["BreakdownTime"]), axis=1)
df["CompleteDatetime"]  = df.apply(lambda row: pd.Timestamp.combine(row["BreakdownDate"], row["CompleteTime"]), axis=1)

df["Downtime"] = (df["CompleteDatetime"] - df["BreakdownDatetime"]).dt.total_seconds() / 60  # Downtime in minutes
df["Uptime"]   = df["BreakdownDatetime"].shift(-1) - df["CompleteDatetime"]
df["Uptime"]   = df["Uptime"].dt.total_seconds() / 60  # Uptime in minutes
df = df.dropna(subset=["Uptime"])  # Remove the last NaN value

# Fit the uptime and downtime data to Weibull distribution
shape_u, loc_u, scale_u = weibull_min.fit(df["Uptime"], floc=0)
shape_d, loc_d, scale_d = weibull_min.fit(df["Downtime"], floc=0)

# Group by MachineID and fit the Weibull distribution for each group
result = []

for machine_id, group in df.groupby("MachineID"):
    shape_u, loc_u, scale_u = weibull_min.fit(group["Uptime"], floc=0)
    shape_d, loc_d, scale_d = weibull_min.fit(group["Downtime"], floc=0)
    
    operation_ref = group["OperationRef"].iloc[0]
    machine_ref = group["MachineRef"].iloc[0]
    
    parameter_uptime = f"({shape_u}, {loc_u}, {scale_u})"
    parameter_downtime = f"({shape_d}, {loc_d}, {scale_d})"
    
    result.append([machine_id, operation_ref, machine_ref, parameter_uptime, parameter_downtime])

# Create a new DataFrame with the results
result_df = pd.DataFrame(result, columns=["MachineID", "OperationRef", "MachineRef", "ParameterUpTime", "ParameterDownTime"])

# Write the result to a new sheet in the Excel file
with pd.ExcelWriter(r"D:\D\University\4th year\Thesis\Processed Data\DataMaster.xlsx", mode='a', engine='openpyxl') as writer:
    result_df.to_excel(writer, sheet_name="Distribution", index=False)
